In [ ]:
import numpy as np

In [ ]:
class Medaline_NN():
  
  def __init__(self,n_layers,layers,learning_rate,num_epocs):
    self.n_layers = 0
    self.layers = []
    self.learning_rate = 1
    self.W = []
    self.num_epocs = 100
    self.n_layers = n_layers
    self.layers = layers
    self.learning_rate = learning_rate
    self.num_epocs = num_epocs
    for i in range(n_layers-1):
      self.W.append(self.random_init((self.layers[i+1], self.layers[i]+1)))
  def Adaline_update(self,index,X,y_actual,t,w):
    '''
      input:
        index: min index
        X: input array X
        y_actual: 1-D array y
        w: weights
    '''
    Q = np.dot(w,X.T)
    if (y_actual==1):
      w = w + self.learning_rate*(1 - Q)*X
    elif (y_actual==-1):
      w = w + self.learning_rate*(-1 - Q)*X
    self.W[index[0]][index[1]] = w
  def threshold(self, X):
    '''
      input: input array X
      checks values and according to threshold assigns value +1 for positive values (>=0) and -1 for negative values (<0)
    '''
    z = np.zeros((X.shape[0]))
    for i in range(len(z)):
      if X[i]>=0:
        z[i]=1
      else:
        z[i] = -1
    return z
  def random_init(self, shape):
        """
        Calculating the initial weights after Random Activation for a particular layer

        Parameters
        ----------
        shape : tuple specifying the shape of the layer for which weights have to be generated

        Returns
        -------
        weight : 1-dimensional numpy array which contains the initial weights for the requested layer
        """
        return np.random.rand(shape[0],shape[1])
  def test(self,train_X,test_X):
    '''
      input:
        train_X: train values X
        test_X: test values X
      prints accuracy
    '''
    acc=0
    count_0 =0
    count_1=0
    for j in range(len(train_X)):
      Y = []
      Z = []
      visited = []
      for i in range(0,self.n_layers-1):
        if (i==0):
          Z.append(np.dot(self.W[i],train_X[j].T))
          Y.append(self.threshold(Z[-1]))
          visited.append(np.ones((Z[-1].shape)))
        else:
          Y[i-1] = np.append(Y[i-1],[1],axis=0)
          Z.append(np.dot(self.W[i],Y[i-1].T))
          Y.append(self.threshold(Z[-1]))
          visited.append(np.ones((Z[-1].shape)))
      if (Y[-1][0] == test_X[j]):
        acc+=1
        if (test_X[j]==-1):
          count_0+=1
        else:
          count_1+=1
    print("count of -1/0 = ",count_0," count of 1 = ",count_1)
    print(acc / len(train_X)*100)
  def find_min(self, Z , visited):
    '''
      input:
        Z: input array
        visited: visited array to see which are already counted
      output:
        return index of min element which has least absolute value of z
    '''
    mini = 1000000
    n = (-1,-1)
    for k in range(len(Z)):
      for l in range(len(Z[k])):
        if (mini>abs(Z[k][l]) and visited[k][l]==1):
          mini = abs(Z[k][l])
          n = (k,l)
    return n
  def forward(self, train_X , test_X):
    '''
      input:
        train_X: train values X
        test_X: test values X
      prints accuracy and updates weights ( basically trains the model)
    '''
    for k in range(self.num_epocs):
      count_0 = 0
      acc=0
      count_1 = 0
      # print(train_X.shape, test_X.shape)
      xt = np.hstack((train_X,test_X.reshape((test_X.shape[0],1))))
      # print(xt.shape)
      np.random.shuffle(xt)
      train_X = xt[:,:3]
      test_X = xt[:,3:]
      test_X = test_X.reshape((test_X.shape[0],))
      for j in range(len(train_X)):
        Y = []
        Z = []
        visited = []
        for i in range(0,self.n_layers-1):
          if (i==0):
            Z.append(np.dot(self.W[i],train_X[j].T))
            Y.append(self.threshold(Z[-1]))
            visited.append(np.ones((Z[-1].shape)))
          else:
            Y[i-1] = np.append(Y[i-1],[1],axis=0)
            Z.append(np.dot(self.W[i],Y[i-1].T))
            Y.append(self.threshold(Z[-1]))
            visited.append(np.ones((Z[-1].shape)))
        if (Y[-1][0] == test_X[j]):
          acc+=1
          if (test_X[j]==-1):
            count_0+=1
          else:
            count_1+=1
        if (Y[-1][0]!= test_X[j]):
          change=0
          while(change!=1):
            y_change= []
            z_change = []
            index = self.find_min(Z,visited)
            if (index[0]==-1):
              print("no change")
              break;
            visited[index[0]][index[1]]=0
            Y[index[0]][index[1]] = -1* Y[index[0]][index[1]]
            for o in range(index[0]+1,self.n_layers-1):
              z_change.append(np.dot(self.W[o],Y[o-1].T))
              y_change.append(self.threshold(z_change[-1]))
            Y[index[0]][index[1]] = -1* Y[index[0]][index[1]]
            
            if (len(y_change)==0):
              # print(index, Y[index[0]-1],self.W[index[0]][index[1]],sep ="\n")
              self.Adaline_update(index,Y[index[0]-1],test_X[j],-1*test_X[j],self.W[index[0]][index[1]])
              change=1
            else:
              if (y_change[-1][0] == test_X[j]):
                if(index[0]!=0):
                  # print(index, Y[index[0]-1],self.W[index[0]][index[1]],sep ="\n")
                  self.Adaline_update(index,Y[index[0]-1],test_X[j],-1*test_X[j],self.W[index[0]][index[1]])
                  change=1
                elif(index[0]==0):
                  # print(index, Y[index[0]-1],self.W[index[0]][index[1]],sep ="\n")
                  self.Adaline_update(index,train_X[j],test_X[j],-1*test_X[j],self.W[index[0]][index[1]])
                  change=1
      print("count of -1/0 = ",count_0," count of 1 = ",count_1)
      print("Accuracy => ",acc/len(train_X)*100)

In [ ]:
X = []
Y = []
#make sample data points
for i in range(36):
  x_i = -1+0.25*i
  for j in range(36):
    x_j = -1 +0.25*j
    X.append([x_i,x_j])
    if (x_i>4 and x_i<6 and x_j>4 and x_j<4):
      Y.append(-1)
    elif (x_i>=2 and x_i<=4 and x_j>=0 and x_j<=6):
      Y.append(1)
    elif (x_i>=0 and x_i<=6 and x_j>=2 and x_j<=4):
      Y.append(1)
    else:
      Y.append(-1)

In [ ]:
#sample points
X = [[-1, -1], [-1, 0], [-1, 1], [-1, 2], [-1, 3], [-1, 4], [-1, 5], [-1, 6], [-1, 7], [0, -1], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [1, -1], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [2, -1], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [3, -1], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [4, -1], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7], [5, -1], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [6, -1], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [7, -1], [7, 0], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5], [7, 6], [7, 7]]
Y = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
x_train = np.asarray(X)
x_train = np.c_[X,np.ones(len(X))]
y_train = np.asarray(Y)
#output received
print(len(x_train),"count of 1",Y.count(1),"count of -1",Y.count(-1),sep = " ")
for i in range(len(x_train)):
  print(*x_train[i],y_train[i])

81 count of 1 32 count of -1 49
-1.0 -1.0 1.0 -1
-1.0 0.0 1.0 -1
-1.0 1.0 1.0 -1
-1.0 2.0 1.0 -1
-1.0 3.0 1.0 -1
-1.0 4.0 1.0 -1
-1.0 5.0 1.0 -1
-1.0 6.0 1.0 -1
-1.0 7.0 1.0 -1
0.0 -1.0 1.0 -1
0.0 0.0 1.0 -1
0.0 1.0 1.0 -1
0.0 2.0 1.0 1
0.0 3.0 1.0 1
0.0 4.0 1.0 1
0.0 5.0 1.0 -1
0.0 6.0 1.0 -1
0.0 7.0 1.0 -1
1.0 -1.0 1.0 -1
1.0 0.0 1.0 -1
1.0 1.0 1.0 -1
1.0 2.0 1.0 1
1.0 3.0 1.0 1
1.0 4.0 1.0 1
1.0 5.0 1.0 -1
1.0 6.0 1.0 -1
1.0 7.0 1.0 -1
2.0 -1.0 1.0 -1
2.0 0.0 1.0 1
2.0 1.0 1.0 1
2.0 2.0 1.0 1
2.0 3.0 1.0 1
2.0 4.0 1.0 1
2.0 5.0 1.0 1
2.0 6.0 1.0 1
2.0 7.0 1.0 -1
3.0 -1.0 1.0 -1
3.0 0.0 1.0 1
3.0 1.0 1.0 1
3.0 2.0 1.0 1
3.0 3.0 1.0 -1
3.0 4.0 1.0 1
3.0 5.0 1.0 1
3.0 6.0 1.0 1
3.0 7.0 1.0 -1
4.0 -1.0 1.0 -1
4.0 0.0 1.0 1
4.0 1.0 1.0 1
4.0 2.0 1.0 1
4.0 3.0 1.0 1
4.0 4.0 1.0 1
4.0 5.0 1.0 1
4.0 6.0 1.0 1
4.0 7.0 1.0 -1
5.0 -1.0 1.0 -1
5.0 0.0 1.0 -1
5.0 1.0 1.0 -1
5.0 2.0 1.0 1
5.0 3.0 1.0 1
5.0 4.0 1.0 1
5.0 5.0 1.0 -1
5.0 6.0 1.0 -1
5.0 7.0 1.0 -1
6.0 -1.0 1.0 -1
6.0 0.0 1.0 -1
6.0 1

In [ ]:
#Initialise the neural network 
nn = Medaline_NN(3,[len(x_train[0])-1,8,2,1],0.01,3000)
nn.forward(x_train,y_train)
nn.test( x_train,y_train)

Streaming output truncated to the last 5000 lines.
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.20987654320987
count of -1/0 =  3  count of 1 =  32
Accuracy =>  43.209876543209

In [ ]:
#Save model
import pickle
pickle.dump(nn, open("Madaline_2nd_8+2 hidden","wb"))

In [ ]:
best_model = pickle.load(open("Madaline_2nd_8+2 hidden","rb"))
best_model.test(x_train,y_train)

count of -1/0 =  4  count of 1 =  32
44.44444444444444
